In [1]:
import pandas as pd 
import tensorflow as tf
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
import random
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import json

In [2]:
dataset_path = r"D:\\CSPC_LIFE\\3RD YEAR\\FIRST_SEMESTER\\Application Delopment and Emerging Technologies\\AskCrack\\dataset_and_model\\wall_surface_dataset"

valid_extensions = ('.jpeg', '.jpg', '.png', '.JPG', '.JPEG', '.PNG')

pos_image_paths = []
neg_image_paths = []

for root, dirs, files in os.walk(dataset_path):
    folder = os.path.basename(root).lower()
    
    if folder in ["positive", "negative"]:
        for file in files:
            if file.endswith(valid_extensions):
                full_path = os.path.join(root, file)
                
                if folder == "positive":
                    pos_image_paths.append(full_path)
                elif folder == "negative":
                    neg_image_paths.append(full_path)

# Balance the dataset
min_samples = min(len(pos_image_paths), len(neg_image_paths))
print(f"Original - Positive: {len(pos_image_paths)}, Negative: {len(neg_image_paths)}")
print(f"Using {min_samples} samples from each class")

# Randomly sample to balance
random.seed(42)
pos_sample = random.sample(pos_image_paths, min_samples)
neg_sample = random.sample(neg_image_paths, min_samples)

# Combine and create DataFrame
image_paths = pos_sample + neg_sample
labels = [1] * len(pos_sample) + [0] * len(neg_sample)

df = pd.DataFrame({
    'images': image_paths,
    'label': labels
})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Balanced dataset - Total: {len(df)}, Positives: {sum(df['label']==1)}, Negatives: {sum(df['label']==0)}")


Original - Positive: 3555, Negative: 20000
Using 3555 samples from each class
Balanced dataset - Total: 7110, Positives: 3555, Negatives: 3555


In [3]:
def preprocess_image(path, target_size=(128, 128)):
    """
    Preprocess image for MobileNet.
    IMPORTANT: Use the same function during inference!
    """
    try:
        img = cv2.imread(path)
        if img is None:
            raise ValueError(f"Image not found: {path}")
        
        # Convert BGR to RGB (cv2 loads as BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Resize
        img_resized = cv2.resize(img, target_size)
        
        # Convert to float32
        img_array = img_resized.astype('float32')
        
        # Use MobileNet's preprocessing (scales to [-1, 1])
        img_array = preprocess_input(img_array)
        
        return img_array
    except Exception as e:
        print(f"Error processing {path}: {e}")
        return None

# Process all images
processed_images = []
valid_labels = []

print("\nProcessing images...")
for idx, (path, label) in enumerate(zip(df['images'], df['label'])):
    img_array = preprocess_image(path, target_size=(128, 128))
    if img_array is not None:
        processed_images.append(img_array)
        valid_labels.append(label)
    
    if (idx + 1) % 500 == 0:
        print(f"Processed {idx + 1}/{len(df)} images")

X = np.array(processed_images, dtype='float32')
y = np.array(valid_labels)

print(f"\nFinal shapes - X: {X.shape}, y: {y.shape}")
print(f"X value range: [{X.min():.3f}, {X.max():.3f}]")



Processing images...


Processed 500/7110 images
Processed 1000/7110 images
Processed 1500/7110 images
Processed 2000/7110 images
Processed 2500/7110 images
Processed 3000/7110 images
Processed 3500/7110 images
Processed 4000/7110 images
Processed 4500/7110 images
Processed 5000/7110 images
Processed 5500/7110 images
Processed 6000/7110 images
Processed 6500/7110 images
Processed 7000/7110 images

Final shapes - X: (7110, 128, 128, 3), y: (7110,)
X value range: [-1.000, 1.000]


In [4]:
# First split: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Split train into train and validation: 75% train, 25% validation of the 80%
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"\nTrain: {len(y_train)} (Pos: {sum(y_train==1)}, Neg: {sum(y_train==0)})")
print(f"Val: {len(y_val)} (Pos: {sum(y_val==1)}, Neg: {sum(y_val==0)})")
print(f"Test: {len(y_test)} (Pos: {sum(y_test==1)}, Neg: {sum(y_test==0)})")



Train: 4550 (Pos: 2275, Neg: 2275)
Val: 1138 (Pos: 569, Neg: 569)
Test: 1422 (Pos: 711, Neg: 711)


In [5]:
base_model = MobileNet(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base initially
base_model.trainable = False

# Add custom top with more regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile with class weights to handle any residual imbalance
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 64, 64, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 64, 64, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 64, 64, 64)     │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 65, 65, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 32, 32, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 32, 32, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 32, 32, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 32, 32, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,360,193 (12.82 MB)

 Trainable params: 131,329 (513.00 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [6]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

print("\n" + "="*60)
print("PHASE 1: Training with frozen base")
print("="*60)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)



PHASE 1: Training with frozen base
Epoch 1/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 118s 701ms/step - accuracy: 0.9831 - loss: 0.8214 - precision: 0.9803 - recall: 0.9859 - val_accuracy: 1.0000 - val_loss: 0.2272 - val_precision: 1.0000 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 104s 726ms/step - accuracy: 0.9936 - loss: 0.1570 - precision: 0.9934 - recall: 0.9938 - val_accuracy: 0.9991 - val_loss: 0.0868 - val_precision: 0.9982 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 104s 727ms/step - accuracy: 0.9954 - loss: 0.0735 - precision: 0.9952 - recall: 0.9956 - val_accuracy: 0.9991 - val_loss: 0.0461 - val_precision: 1.0000 - val_recall: 0.9982 - learning_rate: 0.0010
Epoch 4/20
143/143 ━━━━━━━━━━━━━━━━━━━━ 154s 813ms/step - accuracy: 0.9934 - loss: 0.0636 - precision: 0.9934 - recall: 0.9934 - val_accuracy: 0.9991 - val_loss: 0.0462 - val_precision: 1.0000 - val_recall: 0.9982 - learning_rate: 0.0010
Epoch 5/20
1

In [7]:
base_model.trainable = True

# Freeze early layers, only train last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("\n" + "="*60)
print("PHASE 2: Fine-tuning")
print("="*60)

history_fine = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)



PHASE 2: Fine-tuning
Epoch 1/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.9492 - loss: 0.1370 - precision_1: 0.9091 - recall_1: 0.9982 - val_accuracy: 0.9991 - val_loss: 0.0175 - val_precision_1: 0.9982 - val_recall_1: 1.0000 - learning_rate: 1.0000e-05
Epoch 2/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.9936 - loss: 0.0360 - precision_1: 0.9917 - recall_1: 0.9956 - val_accuracy: 1.0000 - val_loss: 0.0161 - val_precision_1: 1.0000 - val_recall_1: 1.0000 - learning_rate: 1.0000e-05
Epoch 3/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.9936 - loss: 0.0345 - precision_1: 0.9917 - recall_1: 0.9956 - val_accuracy: 1.0000 - val_loss: 0.0157 - val_precision_1: 1.0000 - val_recall_1: 1.0000 - learning_rate: 1.0000e-05
Epoch 4/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.9974 - loss: 0.0248 - precision_1: 0.9969 - recall_1: 0.9978 - val_accuracy: 1.0000 - val_loss: 0.0153 - val_precision_1: 1.0000 - val_recall_1: 1.0000 - learning_rate: 1.000

In [8]:
print("\n" + "="*60)
print("EVALUATION ON TEST SET")
print("="*60)

y_pred_proba = model.predict(X_test, verbose=0)

# Analyze prediction distribution
print(f"\nPrediction probability statistics:")
print(f"Mean: {y_pred_proba.mean():.4f}")
print(f"Std: {y_pred_proba.std():.4f}")
print(f"Min: {y_pred_proba.min():.4f}")
print(f"Max: {y_pred_proba.max():.4f}")

# Find optimal threshold
from sklearn.metrics import f1_score, precision_score, recall_score

thresholds = np.arange(0.1, 0.9, 0.05)
best_threshold = 0.5
best_f1 = 0
threshold_results = []

for thresh in thresholds:
    y_pred = (y_pred_proba > thresh).astype(int).flatten()
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    threshold_results.append({
        'threshold': thresh,
        'f1': f1,
        'precision': precision,
        'recall': recall
    })
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thresh

print(f"\n{'Threshold':<12} {'F1':<8} {'Precision':<12} {'Recall':<8}")
print("-" * 45)
for res in threshold_results:
    marker = " <-- BEST" if res['threshold'] == best_threshold else ""
    print(f"{res['threshold']:<12.2f} {res['f1']:<8.4f} {res['precision']:<12.4f} {res['recall']:<8.4f}{marker}")

# Final evaluation with best threshold
print(f"\n{'='*60}")
print(f"FINAL RESULTS (Threshold: {best_threshold:.2f})")
print(f"{'='*60}")

y_pred_final = (y_pred_proba > best_threshold).astype(int).flatten()

print("\nClassification Report:")
print(classification_report(y_test, y_pred_final, target_names=['Negative', 'Positive']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred_final)
print(f"                Predicted")
print(f"              Neg    Pos")
print(f"Actual  Neg   {cm[0][0]:<6} {cm[0][1]:<6}")
print(f"        Pos   {cm[1][0]:<6} {cm[1][1]:<6}")



EVALUATION ON TEST SET

Prediction probability statistics:
Mean: 0.5009
Std: 0.4979
Min: 0.0000
Max: 1.0000

Threshold    F1       Precision    Recall  
---------------------------------------------
0.10         0.9979   0.9958       1.0000   <-- BEST
0.15         0.9979   0.9958       1.0000  
0.20         0.9979   0.9958       1.0000  
0.25         0.9979   0.9958       1.0000  
0.30         0.9979   0.9958       1.0000  
0.35         0.9979   0.9958       1.0000  
0.40         0.9979   0.9958       1.0000  
0.45         0.9979   0.9958       1.0000  
0.50         0.9972   0.9958       0.9986  
0.55         0.9972   0.9958       0.9986  
0.60         0.9972   0.9958       0.9986  
0.65         0.9972   0.9958       0.9986  
0.70         0.9965   0.9958       0.9972  
0.75         0.9958   0.9958       0.9958  
0.80         0.9972   0.9986       0.9958  
0.85         0.9965   0.9986       0.9944  

FINAL RESULTS (Threshold: 0.10)

Classification Report:
              precision    rec

In [9]:
# ============================================
# 8. Save model directly as TFLite ONLY
# ============================================
import tensorflow as tf

print("\nConverting model directly to TFLite...")

# Convert trained Keras model in memory → TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (Optional: enable this if you want smaller file size)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save TFLite model
tflite_path = "mobilenet_wall_crack_model.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"✓ Saved TFLite model as: {tflite_path}")
print("==============================================")
print(" TRAINING COMPLETE — TFLITE MODEL SAVED")
print("==============================================")



Converting model directly to TFLite...
INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp7lh2ttb6\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp7lh2ttb6\assets


Saved artifact at 'C:\Users\Admin\AppData\Local\Temp\tmp7lh2ttb6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2407397368912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397373520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397373904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397373136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397372368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397373712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397374096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397374864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397374480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2407397372176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  24073